In [1]:
import pandas as pd
from datetime import datetime

import rqalpha as rqa
import rqalpha_plus

import rqdatac as rqd
rqd.init()

import rqoptimizer as rqo

# ==========================生成目标组合===========================
    
def get_target_stock(date,benchmark):
    """选取当期中证500融券股票"""
    # 融券股票列表
    margin_stocks = rqd.get_margin_stocks(date=date,exchange=None,margin_type='stock')
     # 中证500成分股
    component = rqd.index_components(benchmark, date)
    # 取交集
    target_stock = list(set(component).intersection(margin_stocks))
    return target_stock


def get_target_portfolio(target_stock,date,benchmark):
    """按照产品设置, 计算目标组合"""
    # 定义目标函数和约束条件
    objective = rqo.MinTrackingError()
    
    # 行业中性
    industry_constraints = rqo.WildcardIndustryConstraint(
                                exclude=None,
                                lower_limit=-0.1,
                                upper_limit=0.1,
                                relative=True,
                                hard=True,
                                classification=rqo.IndustryClassification.ZX)
    # 风格约束
    style_constraints = rqo.StyleConstraint(
                                ['size','beta','non_linear_size'], 
                                lower_limit=-0.1, 
                                upper_limit=0.1, 
                                relative=True, 
                                hard=True)
    
    constraints = [industry_constraints, style_constraints]
    # 个股权重
    bnds = {}
    index_weights = rqd.index_weights(benchmark,date)
    for code in target_stock:
        if code in index_weights.index:
            stock_weight = index_weights.loc[code]
            bnds[code] = (max(0,stock_weight-0.05),stock_weight+0.05)
    
    target_portfolio = rqo.portfolio_optimize(target_stock,date, 
                                       benchmark=benchmark, 
                                       objective=objective,
                                       cons=constraints,
                                       bnds=bnds) 
    # 由于精度问题, 优化结果可能包含非常接近0的股票权重, 在此直接舍去
    return target_portfolio.loc[lambda x:x!=0]


def generate_trade_records(current_position, target_portfolio, capital, datetime, round_to_lots=True):
    datetime = pd.to_datetime(datetime)
    
    union = target_portfolio.index.union(current_position.index)
    current_position = current_position.reindex(union, fill_value=0)
    target_portfolio = target_portfolio.reindex(union, fill_value=0)
    
    price = rqd.get_price(union, start_date=datetime, end_date=datetime, fields=['close']).squeeze()
    
    target_position = target_portfolio * capital / price
    
    trade_records = pd.DataFrame(index=union.rename('order_book_id'))
    trade_records['datetime'] = datetime
    
    if round_to_lots:
        _round = lambda x: round(x / 100) * 100
    else:
        _round = lambda x: x.round(0)
    quantity_to_trade = _round(target_position - current_position)
    trade_records['last_quantity'] = quantity_to_trade.abs()
    
    compute_side = lambda q: 'BUY' if q > 0 else 'SELL'
    trade_records['side'] = quantity_to_trade.apply(compute_side)
    
    trade_records['last_price'] = price
    trade_records['transaction_cost'] = 0
    
    trade_records = trade_records[trade_records['last_quantity'] != 0]
    
    return trade_records.reset_index()

# 获得今日目标权重

In [2]:
date = '20200819'
benchmark= '000905.XSHG'

In [3]:
target_stock = get_target_stock(date,benchmark)

In [4]:
target_portfolio = get_target_portfolio(target_stock,date,benchmark)

WARN: aa_init returned NULL, no acceleration applied.


# 昨日持仓数据

In [23]:
capital

170801120.63

In [21]:
current_position = pd.read_excel(r'股票-持仓20200819.xlsx',index_col=0)
current_position['资产代码'] = current_position['资产代码'].apply(lambda x:x[:6]+'.XSHG' if x[0]=='6' else x[:6]+'.XSHE')
current_position = current_position.set_index('资产代码')['持仓数量']
# 昨日收盘价
close = rqd.get_price(current_position.index.tolist(),'20200818','20200818',fields='close').squeeze()
# 总资金
capital = (current_position*close).sum()

# 今日调仓股票及数量

In [24]:
result = generate_trade_records(current_position,target_portfolio,capital,'20200818')

In [25]:
result

,order_book_id,datetime,last_quantity,side,last_price,transaction_cost
0,000008.XSHE,2020-08-18,18700.0,SELL,3.17,0
1,000009.XSHE,2020-08-18,83000.0,BUY,8.27,0
2,000012.XSHE,2020-08-18,15400.0,SELL,6.27,0
3,000021.XSHE,2020-08-18,6600.0,SELL,26.75,0
4,000025.XSHE,2020-08-18,4400.0,BUY,22.70,0
...,...,...,...,...,...,...
459,603882.XSHG,2020-08-18,1000.0,SELL,89.42,0
460,603883.XSHG,2020-08-18,3500.0,BUY,104.28,0
461,603885.XSHG,2020-08-18,5900.0,SELL,10.40,0
462,603888.XSHG,2020-08-18,6100.0,BUY,27.60,0


In [28]:
result.to_csv(r'trades_20200819.csv',index=False)